In [2]:
import pandas as pd
import re
from io import StringIO

pattern = re.compile(r"^20\d{2}-\d{2}")
clean_lines = []
expected_cols = None

with open("data.csv", "r", encoding="utf-8") as f:
    lines = f.readlines()

# 1️⃣ Conserver la première ligne (l’en-tête)
header = lines[0]
clean_lines.append(header)

# 2️⃣ Filtrer uniquement les lignes valides commençant par une date
for line in lines[1:]:
    if pattern.match(line.strip()):
        n_cols = line.count(";") + 1
        if expected_cols is None:
            expected_cols = header.count(";") + 1
        if n_cols == expected_cols:
            clean_lines.append(line)

# 3️⃣ Charger dans pandas
df = pd.read_csv(StringIO("".join(clean_lines)), sep=";")

print(f"{len(df)} lignes valides chargées.")

df.info()


10332 lignes valides chargées.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10332 entries, 0 to 10331
Data columns (total 26 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   Date                                                                                         10332 non-null  object 
 1   Service                                                                                      10332 non-null  object 
 2   Gare de départ                                                                               10332 non-null  object 
 3   Gare d'arrivée                                                                               10332 non-null  object 
 4   Durée moyenne du trajet                                                                      10332 non-null  int64  
 5   N

In [3]:
df = df.drop(columns=['Commentaire annulations', 'Commentaire retards au départ'])
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10332 entries, 0 to 10331
Data columns (total 24 columns):
 #   Column                                                                                       Non-Null Count  Dtype         
---  ------                                                                                       --------------  -----         
 0   Date                                                                                         10332 non-null  datetime64[ns]
 1   Service                                                                                      10332 non-null  object        
 2   Gare de départ                                                                               10332 non-null  object        
 3   Gare d'arrivée                                                                               10332 non-null  object        
 4   Durée moyenne du trajet                                                                      10332 non-null  int

In [4]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Filtrer les lignes pour ne garder que celles où toutes les valeurs numériques sont >= 0
df_clean = df[(df[num_cols] >= 0).all(axis=1)]
df = df_clean

In [25]:
import plotly.express as px

# -----------------------------
# -----------------------------
# 3️⃣ Distribution des retards (Histogramme interactif)
# 
df_monthly = (
    df.groupby('Date', as_index=False)['Nombre de trains en retard au départ']
    .sum()
    .sort_values('Date')
)
fig_hist = px.bar(
    df_monthly, 
    y='Nombre de trains en retard au départ', 
    x='Date'
)
fig_hist.show()




In [26]:
fig = px.bar(
    df.groupby('Date', as_index=False)['Nombre de trains annulés'].sum(),
    x='Date',
    y='Nombre de trains annulés',
    title='Nombre de trains annulés par mois'
)

fig.show()


In [27]:
# Calculer le retard total par ligne (Départ + Arrivée)
df_retards = (
    df.groupby(['Gare de départ', 'Gare d\'arrivée'], as_index=False)['Retard moyen de tous les trains à l\'arrivée']
    .mean()
    .sort_values('Retard moyen de tous les trains à l\'arrivée', ascending=False)
    .head(10)
)

# Graphique des 10 lignes avec le plus de retard moyen
fig = px.bar(
    df_retards,
    x='Retard moyen de tous les trains à l\'arrivée',
    y=df_retards['Gare de départ'] + " → " + df_retards['Gare d\'arrivée'],
    orientation='h',
    title='🚆 Top 10 des lignes avec le plus de retard moyen à l’arrivée'
)

fig.show()


In [28]:
fig1 = px.line(
    df.groupby('Date', as_index=False)['Retard moyen de tous les trains à l\'arrivée'].mean(),
    x='Date',
    y='Retard moyen de tous les trains à l\'arrivée',
    title='Évolution du retard moyen à l’arrivée (tous services confondus)',
    markers=True
)
fig1.show()


In [29]:
cols_causes = [
    'Prct retard pour causes externes',
    'Prct retard pour cause infrastructure',
    'Prct retard pour cause gestion trafic',
    'Prct retard pour cause matériel roulant',
    'Prct retard pour cause gestion en gare et réutilisation de matériel',
    'Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)'
]

# Moyenne de chaque cause sur l'ensemble du dataset
mean_causes = df[cols_causes].mean().reset_index()
mean_causes.columns = ['Cause', 'Pourcentage']

fig5 = px.pie(
    mean_causes,
    names='Cause',
    values='Pourcentage',
    title='Répartition moyenne des causes de retard'
)
fig5.show()


In [ ]:
# Moyenne des retards par mois en fonction de la gae

In [11]:
import pandas as pd


# Extraire les gares de départ et d'arrivée
gares_depart = df["Gare de départ"].unique()
gares_arrivee = df["Gare d'arrivée"].unique()

# Fusionner les deux listes et supprimer les doublons
toutes_gares = sorted(set(gares_depart) | set(gares_arrivee))

# Convertir en DataFrame
gares_df = pd.DataFrame(toutes_gares, columns=["Gare"])
toutes_gares

['AIX EN PROVENCE TGV',
 'ANGERS SAINT LAUD',
 'ANGOULEME',
 'ANNECY',
 'ARRAS',
 'AVIGNON TGV',
 'BARCELONA',
 'BELLEGARDE (AIN)',
 'BESANCON FRANCHE COMTE TGV',
 'BORDEAUX ST JEAN',
 'BREST',
 'CHAMBERY CHALLES LES EAUX',
 'DIJON VILLE',
 'DOUAI',
 'DUNKERQUE',
 'FRANCFORT',
 'GENEVE',
 'GRENOBLE',
 'ITALIE',
 'LA ROCHELLE VILLE',
 'LAUSANNE',
 'LAVAL',
 'LE CREUSOT MONTCEAU MONTCHANIN',
 'LE MANS',
 'LILLE',
 'LYON PART DIEU',
 'MACON LOCHE',
 'MADRID',
 'MARNE LA VALLEE',
 'MARSEILLE ST CHARLES',
 'METZ',
 'MONTPELLIER',
 'MULHOUSE VILLE',
 'NANCY',
 'NANTES',
 'NICE VILLE',
 'NIMES',
 'PARIS EST',
 'PARIS LYON',
 'PARIS MONTPARNASSE',
 'PARIS NORD',
 'PARIS VAUGIRARD',
 'PERPIGNAN',
 'POITIERS',
 'QUIMPER',
 'REIMS',
 'RENNES',
 'SAINT ETIENNE CHATEAUCREUX',
 'ST MALO',
 'ST PIERRE DES CORPS',
 'STRASBOURG',
 'STUTTGART',
 'TOULON',
 'TOULOUSE MATABIAU',
 'TOURCOING',
 'TOURS',
 'VALENCE ALIXAN TGV',
 'VANNES',
 'ZURICH']

In [12]:
path = "C:/Users/grego/Documents/GitHub/ST2DVZ-SNCF-Streamlit-Project/data/locations.csv"
df2 = pd.read_csv(path, sep=",")
df2


,Gare,lat,lon
0,AIX EN PROVENCE TGV,43.455607,5.316865
1,ANGERS SAINT LAUD,47.464550,-0.554549
2,ANGOULEME,45.654055,0.167220
3,ANNECY,45.902116,6.121500
4,ARRAS,50.287406,2.781632
5,AVIGNON TGV,43.921796,4.785811
6,BARCELONA,41.379355,2.140149
7,BELLEGARDE (AIN),46.110713,5.825996
8,BESANCON FRANCHE COMTE TGV,47.307814,5.953256
9,BORDEAUX ST JEAN,44.826082,-0.555917
